# Gene target analysis

Cell factory engineering strategies can be used to improve the production of progesterone in our modelled S. cerevisiae strain (iMM904_progesterone.xml).

We investigate if gene knockouts suggested in literature actually have an improving effect on the production of progesterone, and if it would be appropiate to incorporate these gene knockouts in our model.

### Knockout analysis

It has been suggested to knockout ERG5 and ERG6 in order to drive the flux towards the production of progesterone instead of the natural produced steriod ergosterol (Figure 1) (Jiang, Yi-qi, and Jian-ping Lin 2022). However, this can be problematic because ergosterol is an essential compound in the cellular membrane (Jordá, Tania, and Sergi Puig 2020).

![figures/pathway_med_strukturer_v3.png](figures/pathway_med_strukturer_v3.png) 

### References
Jiang, Yi-qi, and Jian-ping Lin. "Recent progress in strategies for steroid production in yeasts." World Journal of Microbiology and Biotechnology 38.6 (2022): 1-14.

Jordá, Tania, and Sergi Puig. "Regulation of ergosterol biosynthesis in Saccharomyces cerevisiae." Genes 11.7 (2020): 795.